In [37]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

/home/taeksoon/anaconda3/envs/dl/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("Step: %d %s" % (step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

### Open AI Gym

In [28]:
env = gym.make('CartPole-v0')

# Run a demo of the environment
obs = env.reset()
img = env.render(mode="rgb_array")
img.shape

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


(400, 600, 3)

In [29]:
env.action_space

Discrete(2)

In [30]:
action = 1
obs, reward, done, info = env.step(action)
print(obs)
print(reward)
print(done)
print(info)

[-0.01659118  0.24160619 -0.0441714  -0.32322813]
1.0
False
{}


### Simple Basic Policy

In [31]:
def basic_policy(obs):
    angle = obs[2]
    # accelerate left if the pole is leaning left, and vice versa
    return 0 if angle < 0 else 1

In [35]:
totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(1000): # 1000 steps max
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
#         show_state(env, step, info)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [36]:
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

(42.076, 8.799671812062083, 24.0, 64.0)

### Neural Network Policy

In [54]:
n_inputs = 4 # == env.observation_space.shape[0]
n_hidden = 4
n_outputs = 1 # probability of acclerating left

learning_rate = 0.01
initializer = tf.contrib.layers.variance_scaling_initializer()

In [61]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu,
                         kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs,
                         kernel_initializer=initializer)
outputs = tf.nn.sigmoid(logits)

p_left_and_right = tf.concat(axis=1, values=[outputs, 1-outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

In [62]:
y = 1. - tf.to_float(action)

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]

gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))

training_op = optimizer.apply_gradients(grads_and_vars_feed)

In [63]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std
            for discounted_rewards in all_discounted_rewards]

In [64]:
n_iterations = 250
n_max_steps = 1000
n_games_per_update = 10
save_iterations = 10
discount_rate = 0.95

In [47]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = []
        all_gradients = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients],
                                                    feed_dict = {X: obs.reshape(1, n_inputs)})
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
            
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean(
                [reward * all_gradients[game_index][step][var_index]
                    for game_index, rewards in enumerate(all_rewards)
                    for step, reward in enumerate(rewards)],
                axis=0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "rl_model/my_policy_net_pg.ckpt")

In [65]:
def pg_policy(obs):
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, "rl_model/my_policy_net_pg.ckpt")
        return sess.run(action, feed_dict={X: obs.reshape(1, n_inputs)})[0][0]

In [ ]:
totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(1000): # 1000 steps max
        fitted_action = pg_policy(obs)
        obs, reward, done, info = env.step(fitted_action)
#         show_state(env, step, info)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)
    
print(np.mean(totals), np.std(totals), np.min(totals), np.max(totals))

INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from rl_model/